### SlurmDriver Example

In [1]:
from slurmflow.driver import SlurmDriver
from slurmflow.config import ConfigParser
from os.path import abspath

The `SlurmDriver` class is capable of interfacting with the __S__imple __L__inux __U__tility for __R__esource __M__anagement. `SlurmDriver` can submit jobs and monitor the queue. 

First we will load a configuration for the `SlurmDriver` using the `ConfigParser` (see `config.ipynb`)

In [2]:
cfg = ConfigParser('config/slurm.yml')
cfg.compile()

{'partition': 'standard',
 'time': '01:00:00',
 'nodes': 1,
 'output_dir': './logs'}

Note that relative paths are used in the configuration file. SLURM prefers absolute paths, so we will use the `get` and `set` methods of the `ConfigParser` to set `output_dir` to its absolute path.

In [3]:
cfg.set('output_dir', abspath(cfg.get('output_dir')))
cfg.compile()

{'partition': 'standard',
 'time': '01:00:00',
 'nodes': 1,
 'output_dir': '/scratch/zt1/project/tiwary-prj/user/lherron/repos/slurm-flow/logs'}

The config is also missing the `job_name` SBATCH directive. We will set it in-memory as well.

In [4]:
cfg.set("job_name", "hello_world")
cfg.compile()

{'partition': 'standard',
 'time': '01:00:00',
 'nodes': 1,
 'output_dir': '/scratch/zt1/project/tiwary-prj/user/lherron/repos/slurm-flow/logs',
 'job_name': 'hello_world'}

As you can probably guess, the submitted command will be `python -c print("Hello World!")`

In [19]:
cmd='python -c "print(\\"Hello World!\\")"'

In [45]:
driver = SlurmDriver(verbose=True)
driver.submit_job(cmd, 
                  slurm_args=cfg.compile(),
                  env='slurmflow', # optionally provide the name of a conda or mamba env (see venv kwarg).
                  modules=['cuda/11.8.0'],
                  track=True # add jobid to the registry
                 ) 

slurmflow - INFO - ['#!/bin/bash', '#SBATCH --partition=standard\n#SBATCH --ntasks=1\n#SBATCH --cpus-per-task=1\n#SBATCH --mem=8G\n#SBATCH --time=01:00:00\n#SBATCH --job-name=hello_world\n#SBATCH --nodes=1\n#SBATCH --output=/scratch/zt1/project/tiwary-prj/user/lherron/repos/slurm-flow/logs/slurm_hello_world.out\n#SBATCH --error=/scratch/zt1/project/tiwary-prj/user/lherron/repos/slurm-flow/logs/slurm_hello_world.err', 'source $MAMBA_ROOT_PREFIX/etc/profile.d/micromamba.sh', 'micromamba activate slurmflow', 'module load cuda/11.8.0', 'python -c "print(\\"Hello World!\\")"']


'4741031'

In [42]:
import importlib
import slurmflow
importlib.reload(slurmflow)

import slurmflow.driver
# import slurmflow.serializer
import slurmflow.config
importlib.reload(slurmflow.driver)

# importlib.reload(slurmflow.serializer)
importlib.reload(slurmflow.config)

from slurmflow.driver import SlurmDriver